<a href="https://colab.research.google.com/github/rkb-1/mrca/blob/master/Kinematic_Analysis_of_RRP_robot_Body_fixed_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Kinematics of Open loop chains - Body-fixed Representation 
-Rohit Kumar (r.kumar@dfki.de)

## Helper functions

In [ ]:
# Import required libraries
import numpy as np
from sympy import Matrix, symbols, Identity, cos, sin, zeros, simplify, init_printing, print_latex

def SE3AdjInvMatrix(C):
    """Compute Inverse of (6x6) Adjoint Matrix for SE(3)

    Args:
        C ([type]): [description]

    Returns:
        sympy.Matrix: Inverse of (6x6) Adjoint Matrix
    """
    AdInv = Matrix([[C[0, 0], C[1, 0], C[2, 0], 0, 0, 0],
                    [C[0, 1], C[1, 1], C[2, 1], 0, 0, 0],
                    [C[0, 2], C[1, 2], C[2, 2], 0, 0, 0],
                    [-C[2, 3]*C[1, 0]+C[1, 3]*C[2, 0], C[2, 3]*C[0, 0]-C[0, 3]*C[2, 0],
                        (-C[1, 3])*C[0, 0]+C[0, 3]*C[1, 0], C[0, 0], C[1, 0], C[2, 0]],
                    [-C[2, 3]*C[1, 1]+C[1, 3]*C[2, 1], C[2, 3]*C[0, 1]-C[0, 3]*C[2, 1],
                        (-C[1, 3])*C[0, 1]+C[0, 3]*C[1, 1], C[0, 1], C[1, 1], C[2, 1]],
                    [-C[2, 3]*C[1, 2]+C[1, 3]*C[2, 2], C[2, 3]*C[0, 2]-C[0, 3]*C[2, 2],
                        (-C[1, 3])*C[0, 2]+C[0, 3]*C[1, 2], C[0, 2], C[1, 2], C[2, 2]]])
    return AdInv

def SE3AdjMatrix(C):
    """Compute (6x6) Adjoint Matrix for SE(3)

    Args:
        C ([type]): [description]

    Returns:
    sympy.Matrix: (6x6) Adjoint Matrix
    """
    Ad = Matrix([[C[0, 0], C[0, 1], C[0, 2], 0, 0, 0],
                 [C[1, 0], C[1, 1], C[1, 2], 0, 0, 0],
                 [C[2, 0], C[2, 1], C[2, 2], 0, 0, 0],
                 [-C[2, 3]*C[1, 0]+C[1, 3]*C[2, 0], -C[2, 3]*C[1, 1]+C[1, 3]*C[2, 1],
                  -C[2, 3]*C[1, 2]+C[1, 3]*C[2, 2], C[0, 0], C[0, 1], C[0, 2]],
                 [C[2, 3]*C[0, 0]-C[0, 3]*C[2, 0],  C[2, 3]*C[0, 1]-C[0, 3]*C[2, 1],
                     C[2, 3]*C[0, 2]-C[0, 3]*C[2, 2], C[1, 0], C[1, 1], C[1, 2]],
                 [-C[1, 3]*C[0, 0]+C[0, 3]*C[1, 0], -C[1, 3]*C[0, 1]+C[0, 3]*C[1, 1],
                  -C[1, 3]*C[0, 2]+C[0, 3]*C[1, 2], C[2, 0], C[2, 1], C[2, 2]]])
    return Ad

def SE3adMatrix(X):
    """Compute (6x6) adjoint Matrix for SE(3) 
        - also known as spatial cross product in the literature.

    Args:
        X ([type]): [description]

    Returns:
        sympy.Matrix: (6x6) adjoint Matrix
    """
    ad = Matrix([[0, -X[2, 0], X[1, 0], 0, 0, 0],
                 [X[2, 0], 0, -X[0, 0], 0, 0, 0],
                 [-X[1, 0], X[0, 0], 0, 0, 0, 0],
                 [0, -X[5, 0], X[4, 0], 0, -X[2, 0], X[1, 0]],
                 [X[5, 0], 0, -X[3, 0], X[2, 0], 0, -X[0, 0]],
                 [-X[4, 0], X[3, 0], 0, -X[1, 0], X[0, 0], 0]])
    return ad

def SE3Exp(XX, t):
    """compute exponential mapping for SE(3).

    Args:
        XX ([type]): [description]
        t ([type]): [description]

    Returns:
        [type]: [description]
    """
    X = XX.T
    xi = Matrix(X[0:3])
    eta = Matrix(X[3:6])
    xihat = Matrix([[0, -xi[2], xi[1]],
                    [xi[2], 0, -xi[0]],
                    [-xi[1], xi[0], 0]])
    R = Matrix(Identity(3)) + sin(t)*xihat + (1-cos(t))*(xihat*xihat)
    if xi == zeros(3, 1):
        p = eta * t
    else:
        p = Matrix(Identity(3)*t + (1-cos(t))*xihat + (t-sin(t))*(xihat*xihat))*eta
    C = R.row_join(p).col_join(Matrix([0, 0, 0, 1]).T)
    return C

def SE3Inv(C):
    """Compute analytical inverse of exponential mapping for SE(3).

    Args:
        C ([type]): [description]

    Returns:
        [type]: [description]
    """
    CInv = Matrix([[C[0, 0], C[1, 0], C[2, 0], -C[0, 0]*C[0, 3]-C[1, 0]*C[1, 3]-C[2, 0]*C[2, 3]],
                   [C[0, 1], C[1, 1], C[2, 1], -C[0, 1] *
                       C[0, 3]-C[1, 1]*C[1, 3]-C[2, 1]*C[2, 3]],
                   [C[0, 2], C[1, 2], C[2, 2], -C[0, 2] *
                       C[0, 3]-C[1, 2]*C[1, 3]-C[2, 2]*C[2, 3]],
                   [0, 0, 0, 1]])
    return CInv

def SO3Exp(x, t):
    """Compute exponential mapping for SO(3) (Rotation Matrix).

    Args:
        x (sympy.Matrix): Rotation axis
        t (double): Rotation angle

    Returns:
        sympy.Matrix: Rotation Matrix
    """
    xhat = Matrix([[0, -x[2, 0], x[1, 0]],
                   [x[2, 0], 0, -x[0, 0]],
                   [-x[1, 0], x[0, 0], 0]])
    R = Matrix(Identity(3)) + sin(t) * xhat + (1-cos(t))*(xhat*xhat)
    return R

def InertiaMatrix(Ixx, Ixy, Ixz, Iyy, Iyz, Izz):
    I = Matrix([[Ixx, Ixy, Ixz],
                [Ixy, Iyy, Iyz],
                [Ixz, Iyz, Izz]])
    return I

def TransformationMatrix(r=Matrix(Identity(3)), t=zeros(3, 1)):
    """Build Transformation matrix from rotation and translation

    Args:
        r (sympy.Matrix): SO(3) Rotation matrix (3,3). Defaults to sympy.Matrix(Identity(3))
        t (sympy.Matrix): Translation vector (3,1). Defaults to sympy.zeros(3,1)

    Returns:
        sympy.Matrix: Transformation matrix
    """
    T = r.row_join(t).col_join(Matrix([[0, 0, 0, 1]]))
    return T

def MassMatrixMixedData(m, Theta, COM):
    """Build mass-inertia matrix in SE(3) from mass, inertia and center of mass information.

    Args:
        m (float): Mass.
        Theta (sympy.Matrix): Inertia.
        COM (sympy.Matrix): Center of Mass.

    Returns:
        sympy.Matrix: Mass-inertia Matrix.
    """
    M = Matrix([[Theta[0, 0], Theta[0, 1], Theta[0, 2], 0, (-COM[2])*m, COM[1]*m],
                [Theta[0, 1], Theta[1, 1], Theta[1, 2],
                    COM[2]*m, 0, (-COM[0]*m)],
                [Theta[0, 2], Theta[1, 2], Theta[2, 2],
                    (-COM[1])*m, COM[0]*m, 0],  
                [0, COM[2]*m, (-COM[1]*m), m, 0, 0],
                [(-COM[2])*m, 0, COM[0]*m, 0, m, 0],
                [COM[1]*m, (-COM[0])*m, 0, 0, 0, m]])
    return M

# Kinematic Analysis of RRP robot

![picture](https://drive.google.com/uc?export=view&id=1JYMjAVi0WXi56IbYKLWqYV_ASSMInpe2)


## Reference configuration of the bodies (body fixed representation) at zero position
A general configuration is given by a rotation matrix $\mathbf{R}$ and position vector $\mathbf{p}$ as follows.
\begin{equation}
        \mathbf{T} = \begin{bmatrix}
                \mathbf{R} & \textbf{p} \\ \mathbf{0} & 1
            \end{bmatrix}
    \end{equation}



In [ ]:
l = symbols("l", real=1)  # link lenghts

p0e = Matrix([0,0,l])
R0e = Matrix(Identity(3))

T0e = TransformationMatrix(R0e,p0e)

print("Zero configuration of end-effector pose: \n")
display(T0e)

Zero configuration of end-effector pose: 



Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, l],
[0, 0, 0, 1]])

## Spatial Representation of Screw axes


In [ ]:
s1_hat = Matrix([1,0,0])
s1_o = Matrix([0,0,0])
display(s1_hat,s1_o)

Matrix([
[1],
[0],
[0]])

Matrix([
[0],
[0],
[0]])

In [ ]:
s2_hat = Matrix([0,1,0])
s2_o = Matrix([0,0,0])
display(s2_hat,s2_o)

Matrix([
[0],
[1],
[0]])

Matrix([
[0],
[0],
[0]])

In [ ]:
s3_hat = Matrix([0,0,1])
display(s3_hat)

Matrix([
[0],
[0],
[1]])

## Body-fixed representation of Screws

In [ ]:
S1 = Matrix([s1_hat, s1_o.cross(s1_hat)])
S2 = Matrix([s2_hat, s2_o.cross(s2_hat)])
S3 = Matrix([Matrix([0,0,0]), s3_hat])
S = S1.row_join(S2).row_join(S3)
print("Screw axes (arranged column wise for the joints in spatial representation :\n")
display(S)

B1 = SE3AdjInvMatrix(T0e)*S1
# display(B1)
B2 = SE3AdjInvMatrix(T0e)*S2
# display(B2)
B3 = SE3AdjInvMatrix(T0e)*S3
# display(B3)
B = B1.row_join(B2).row_join(B3)
print("Screw axes (arranged column wise for the joints in body-fixed representation :\n")
display(B)

Screw axes (arranged column wise for the joints in spatial representation :



Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 1]])

Screw axes (arranged column wise for the joints in body-fixed representation :



Matrix([
[ 1, 0, 0],
[ 0, 1, 0],
[ 0, 0, 0],
[ 0, l, 0],
[-l, 0, 0],
[ 0, 0, 1]])

## Position Analysis

### The product of exponential (POE) formula is given by
\begin{equation}
  {}^0\mathbf T_n(\mathbf q) = {}^0\mathbf T_n(\mathbf 0) \exp([\mathbf B_1]q_1) \exp([\mathbf B_2]q_2) \ldots \exp([\mathbf B_n]q_n)  
  \label{eqn_POE}
 \end{equation}


### Define the generalised coordinates for the system, $q_1$ $q_2$ $q_3$

In [ ]:
q1, q2, q3 = symbols("q1 q2 q3", real=1)  # joint positions
q = Matrix([q1,q2,q3])

T0q = simplify( T0e * SE3Exp(B1,q1) * SE3Exp(B2,q2) * SE3Exp(B3,q3))
print("Forward kinematics : \n")
display(T0q)

Forward kinematics : 



Matrix([
[         cos(q2),       0,          sin(q2),          (l + q3)*sin(q2)],
[ sin(q1)*sin(q2), cos(q1), -sin(q1)*cos(q2), -(l + q3)*sin(q1)*cos(q2)],
[-sin(q2)*cos(q1), sin(q1),  cos(q1)*cos(q2),  (l + q3)*cos(q1)*cos(q2)],
[               0,       0,                0,                         1]])

## Velocity Analysis

In [ ]:
Jb_3 = B3
Jb_2 = SE3AdjMatrix(SE3Exp(-B3,q3)) * B2
Jb_1 = SE3AdjMatrix(SE3Exp(-B3,q3) * SE3Exp(-B2,q2)) * B1
Jb = simplify(Jb_1.row_join(Jb_2).row_join(Jb_3))
print("Body Jacobian: ")
display(Jb)

Body Jacobian: 


Matrix([
[          cos(q2),      0, 0],
[                0,      1, 0],
[          sin(q2),      0, 0],
[                0, l + q3, 0],
[-(l + q3)*cos(q2),      0, 0],
[                0,      0, 1]])

In [ ]:
q1d, q2d, q3d = symbols("qdot1 qdot2 qdot3", real=1)  # joint positions
qd = Matrix([q1d,q2d,q3d])

Vb1 = Jb_1*q1d
Vb2 = Vb1 + Jb_2*q2d
Vb3 = Vb2 + Jb_3*q3d
print("Body Velocities: \n")
display(simplify(Vb1.row_join(Vb2).row_join(Vb3)))
print("\nBody Velocity of end-effector:\n")
Vb = simplify(Jb*qd)
display(Vb)

Body Velocities: 



Matrix([
[          qdot1*cos(q2),           qdot1*cos(q2),           qdot1*cos(q2)],
[                      0,                   qdot2,                   qdot2],
[          qdot1*sin(q2),           qdot1*sin(q2),           qdot1*sin(q2)],
[                      0,          qdot2*(l + q3),          qdot2*(l + q3)],
[-qdot1*(l + q3)*cos(q2), -qdot1*(l + q3)*cos(q2), -qdot1*(l + q3)*cos(q2)],
[                      0,                       0,                   qdot3]])


Body Velocity of end-effector:



Matrix([
[          qdot1*cos(q2)],
[                  qdot2],
[          qdot1*sin(q2)],
[         qdot2*(l + q3)],
[-qdot1*(l + q3)*cos(q2)],
[                  qdot3]])

## Acceleration Analysis

In [ ]:
q1dd, q2dd, q3dd = symbols("qddot1 qddot2 qddot3", real=1)  # joint positions
qdd = Matrix([q1dd,q2dd,q3dd])

 
Jbdot1 = - SE3adMatrix(Vb3) * Jb_1 
Jbdot2 = - SE3adMatrix(Vb3) * Jb_2 + SE3adMatrix(Vb2)*Jb_2 
Jbdot3 = zeros(6,1)

Jbdot = simplify(Jbdot1.row_join(Jbdot2).row_join(Jbdot3))
print("Body Jacobian derivative (Jb_dot)")
display(Jbdot)


Body Jacobian derivative (Jb_dot)


Matrix([
[                        -qdot2*sin(q2),     0, 0],
[                                     0,     0, 0],
[                         qdot2*cos(q2),     0, 0],
[                                     0, qdot3, 0],
[qdot2*(l + q3)*sin(q2) - qdot3*cos(q2),     0, 0],
[                                     0,     0, 0]])

In [ ]:
Vbdot1 = Jb_1*q1dd + Jbdot1 * q1d
Vbdot2 = Vbdot1 + Jb_2*q2dd + Jbdot2 * q2d
Vbdot3 = Vbdot2 + Jb_3*q3dd + Jbdot3 * q3d

print("Body Accelerations: ")
display(simplify(Vbdot1.row_join(Vbdot2).row_join(Vbdot3)))

Vbdot = Jb*qdd + Jbdot*qd
print("\n Body acceleration of end-effector")
display(simplify(Vbdot))

Body Accelerations: 


Matrix([
[                                     qddot1*cos(q2) - qdot1*qdot2*sin(q2),                                      qddot1*cos(q2) - qdot1*qdot2*sin(q2),                                      qddot1*cos(q2) - qdot1*qdot2*sin(q2)],
[                                                                        0,                                                                    qddot2,                                                                    qddot2],
[                                     qddot1*sin(q2) + qdot1*qdot2*cos(q2),                                      qddot1*sin(q2) + qdot1*qdot2*cos(q2),                                      qddot1*sin(q2) + qdot1*qdot2*cos(q2)],
[                                                                        0,                                             qddot2*(l + q3) + qdot2*qdot3,                                             qddot2*(l + q3) + qdot2*qdot3],
[-qddot1*(l + q3)*cos(q2) + qdot1*(qdot2*(l + q3)*sin(q2) - qdot3*cos(q2)), -qddot1


 Body acceleration of end-effector


Matrix([
[                                     qddot1*cos(q2) - qdot1*qdot2*sin(q2)],
[                                                                   qddot2],
[                                     qddot1*sin(q2) + qdot1*qdot2*cos(q2)],
[                                            qddot2*(l + q3) + qdot2*qdot3],
[-qddot1*(l + q3)*cos(q2) + qdot1*(qdot2*(l + q3)*sin(q2) - qdot3*cos(q2))],
[                                                                   qddot3]])

## Relationship with spatial representaion

In [ ]:
Js = SE3AdjMatrix(T0q)*Jb
print("Spatial Jacobian: \n")
display(simplify(Js))

Spatial Jacobian: 



Matrix([
[1,       0,                0],
[0, cos(q1),                0],
[0, sin(q1),                0],
[0,       0,          sin(q2)],
[0,       0, -sin(q1)*cos(q2)],
[0,       0,  cos(q1)*cos(q2)]])

In [ ]:
Vsdot= SE3AdjMatrix(T0q)*Vbdot
print("Spatial acceleration of end-effector")
display(simplify(Vsdot))

Spatial acceleration of end-effector


Matrix([
[                                                                             qddot1],
[                                               qddot2*cos(q1) - qdot1*qdot2*sin(q1)],
[                                               qddot2*sin(q1) + qdot1*qdot2*cos(q1)],
[                                               qddot3*sin(q2) + qdot2*qdot3*cos(q2)],
[-qddot3*sin(q1)*cos(q2) - qdot1*qdot3*cos(q1)*cos(q2) + qdot2*qdot3*sin(q1)*sin(q2)],
[ qddot3*cos(q1)*cos(q2) - qdot1*qdot3*sin(q1)*cos(q2) - qdot2*qdot3*sin(q2)*cos(q1)]])

## Relationship with Analytical expressions or Hybrid Jacobian

In [ ]:
# HYBRID JACOBIAN OR ANALYTICAL JACOBIAN
a = T0q[0:3,0:3].row_join(zeros(3,3))
b = zeros(3,3).row_join(T0q[0:3,0:3])
c = a.col_join(b)
Jh = c*Jb
print("Analytical Jacobian: \n")
display(simplify(Jh))
Vh = simplify(Jh*qd)
print("\n Analytical Velocity of end-effector: \n")
display(Vh)

Analytical Jacobian: 



Matrix([
[                        1,                         0,                0],
[                        0,                   cos(q1),                0],
[                        0,                   sin(q1),                0],
[                        0,          (l + q3)*cos(q2),          sin(q2)],
[-(l + q3)*cos(q1)*cos(q2),  (l + q3)*sin(q1)*sin(q2), -sin(q1)*cos(q2)],
[-(l + q3)*sin(q1)*cos(q2), -(l + q3)*sin(q2)*cos(q1),  cos(q1)*cos(q2)]])


 Analytical Velocity of end-effector: 



Matrix([
[                                                                                   qdot1],
[                                                                           qdot2*cos(q1)],
[                                                                           qdot2*sin(q1)],
[                                                  qdot2*(l + q3)*cos(q2) + qdot3*sin(q2)],
[-qdot1*(l + q3)*cos(q1)*cos(q2) + qdot2*(l + q3)*sin(q1)*sin(q2) - qdot3*sin(q1)*cos(q2)],
[-qdot1*(l + q3)*sin(q1)*cos(q2) - qdot2*(l + q3)*sin(q2)*cos(q1) + qdot3*cos(q1)*cos(q2)]])

In [ ]:
# HYBRID JACOBIAN_dot OR ANALYTICAL JACOBIAN_dot
adjoint_twist = Vh[0:3,:].col_join(zeros(3,1))
Jhdot = c*Jbdot + SE3adMatrix(adjoint_twist) * c * Jb
print("Analytical Jacobian dot: \n")
display(simplify(Jhdot))
Vhdot = Jh*qdd + Jhdot*qd
print("\n Analytical acceleration of end-effector: \n")
display(simplify(Vhdot))

Analytical Jacobian dot: 



Matrix([
[                                                                                 0,                                                                                 0,                                              0],
[                                                                                 0,                                                                    -qdot1*sin(q1),                                              0],
[                                                                                 0,                                                                     qdot1*cos(q1),                                              0],
[                                                                                 0,                                           -qdot2*(l + q3)*sin(q2) + qdot3*cos(q2),                                  qdot2*cos(q2)],
[ qdot1*(l + q3)*sin(q1)*cos(q2) + (qdot2*(l + q3)*sin(q2) - qdot3*cos(q2))*cos(q1),  qdot3*sin(q1)*sin(q2) + (l + q3)*(qdo


 Analytical acceleration of end-effector: 



Matrix([
[                                                                                                                                                                                                                                                                                                                                    qddot1],
[                                                                                                                                                                                                                                                                                                      qddot2*cos(q1) - qdot1*qdot2*sin(q1)],
[                                                                                                                                                                                                                                                                                                      qddot2*sin(q1) + qdot1*qdot2